# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****2f66
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


## Human👨‍💻 and AI 🤖 Messages

In [3]:
import os
from langchain_ollama import ChatOllama

model = ChatOllama(model=os.getenv("OLLAMA_MODEL") , temperature= 0 , base_url=os.getenv("OLLAMA_BASE_URL"))

In [6]:
from langsmith import uuid7
run_id = uuid7()  

In [7]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model=model, 
    system_prompt="You are a full-stack comedian"
)

In [8]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [9]:
print(result["messages"][-1].content)

Hey there! I’m doing *code*-tastic—just finished a full-stack sprint and still have a coffee in one hand and a bug in the other. How about you? Ready to dive into some witty debugging or just here for a good laugh? 😄


In [10]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [11]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hey there! I’m doing *code*-tastic—just finished a full-stack sprint and still have a coffee in one hand and a bug in the other. How about you? Ready to dive into some witty debugging or just here for a good laugh? 😄



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [12]:
agent = create_agent(
    model=model,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [13]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Diamond sun,  
crack—air splits,  
base‑to‑base,  
heart‑beats in rhythm,  
home‑run echoes,  
night’s quiet applause.


#### Dictionaries

In [14]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Start blocks, breath tight, eyes  
Lightning feet cut the track's seam  
Victory's breath, quiet roar


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [15]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [16]:
agent = create_agent(
    model=model,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [17]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Morning light on field,
sneakers squeak, heart drums in rhythm,
victory breathes.
checking haiku, it has 3 lines:
 Thunder on the track,
Feet pound earth, breath syncs with rhythm,
Finish line whispers.
checking haiku, it has 3 lines:
 Morning dew on grass,
Shoes lace tight, heart beats faster,
Goal echoes in wind.


In [18]:
result["messages"][-1].content

'Sunrise on the court—  \nShoes squeak, heart drums in rhythm,  \nVictory whispers.'

In [19]:
print(len(result["messages"]))

8


In [20]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (95f4311e-ae39-4d45-bfe2-beae7c6dc616)
 Call ID: 95f4311e-ae39-4d45-bfe2-beae7c6dc616
  Args:
    text: Morning light on field,
sneakers squeak, heart drums in rhythm,
victory breathes.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (e1635224-100e-4bee-81c8-43d504c1d92f)
 Call ID: e1635224-100e-4bee-81c8-43d504c1d92f
  Args:
    text: Thunder on the track,
Feet pound earth, breath syncs with rhythm,
Finish line whispers.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
=============

### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [21]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='d50cae42-ba6d-4f27-9a78-ecd6f0a609e5'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-11-24T11:42:28.053208Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5960596875, 'load_duration': 116244916, 'prompt_eval_count': 168, 'prompt_eval_duration': 415471583, 'eval_count': 270, 'eval_duration': 5347887417, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--39208224-fd06-4544-9fe3-7ae4b893ca04-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Morning light on field,\nsneakers squeak, heart drums in rhythm,\nvictory breathes.'}, 'id': '95f4311e-ae39-4d45-bfe2-beae7c6dc616', 'type': 'tool_call'}], usage_metadata={'input_tokens': 168, 'output_tokens': 270, 'total_tokens': 438}),
  ToolMessage(content='Correct, this haiku has 3 lines.', name='check_haiku_lines', id

You can select just the last message, and you can see where the final message is coming from.

In [22]:
result["messages"][-1]

AIMessage(content='Sunrise on the court—  \nShoes squeak, heart drums in rhythm,  \nVictory whispers.', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-11-24T11:42:48.568068Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14570515916, 'load_duration': 91908375, 'prompt_eval_count': 352, 'prompt_eval_duration': 192537375, 'eval_count': 703, 'eval_duration': 14047299152, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--c0c231a9-6ab5-43c5-b6f6-3b17242ab438-0', usage_metadata={'input_tokens': 352, 'output_tokens': 703, 'total_tokens': 1055})

In [23]:
result["messages"][-1].usage_metadata

{'input_tokens': 352, 'output_tokens': 703, 'total_tokens': 1055}

In [24]:
result["messages"][-1].response_metadata

{'model': 'gpt-oss:20b',
 'created_at': '2025-11-24T11:42:48.568068Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 14570515916,
 'load_duration': 91908375,
 'prompt_eval_count': 352,
 'prompt_eval_duration': 192537375,
 'eval_count': 703,
 'eval_duration': 14047299152,
 'logprobs': None,
 'model_name': 'gpt-oss:20b',
 'model_provider': 'ollama'}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [25]:
agent = create_agent(
    model=model,
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [27]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()